# Section 1

## Efstathios Galanakis 03112172
## Ioannis Tzanettis 03112506
## Team Number: ???


Datasheet Name: Sonar (Number S3)
Number of features: ???
Number of classes: 2 (Mine, Rock)


This datasheet contains data taken from a sonar signal.
First we use read_csv in order to read the datasheet
We don't have any missing data so we don't need to use any preprocessing technique for
replacing missing values
The label of each samples is placed at the last column
We need to convert labels string values to int values so we are going to create the following mapping:
mapping = {'R' : 0, 'M' : 1}

In [170]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

perc = 0.2

features = []
labels = []


datasheet_path='./small/sonar.all-data'
datasheet = pd.read_csv(datasheet_path, header=None)

labels = datasheet.iloc[:, [-1]]
features = datasheet.iloc[:, 0 :-1].values
# values = datasheet.values
# features = values[:,0:-1]
# labels = values[:,-1]
mapping = {'R' : 0, 'M' : 1}
labels = labels.replace(mapping).values.flatten()
print values.shape
print features.shape, labels.shape


(208, 61)
(208, 60) (208,)


Test data are 20% of the total data and the rest is train data
So we seperate them

In [171]:
x_train, x_test, y_train, y_test = train_test_split(features,labels,test_size=0.2)

print  features.shape, labels.shape
print x_train.shape, x_test.shape, y_train.shape, y_test.shape


(208, 60) (208,)
(166, 60) (42, 60) (166,) (42,)


We use a default MLP classifier in order to compare with our improved

In [172]:
default_clf_Mlp = MLPClassifier();
default_clf_Mlp.fit(x_train, y_train)
results = default_clf_Mlp.predict(x_test)
print classification_report(results, y_test)
print accuracy_score(results,y_test)

             precision    recall  f1-score   support

          0       0.53      0.64      0.58        14
          1       0.80      0.71      0.75        28

avg / total       0.71      0.69      0.70        42

0.690476190476


## Check Dataset
We have to check if we have devided correctly our dataset in train and test data

In [173]:
print  

## Feature selection
Now it's time to create our improved MLP classifier. First we create a VarianceThreshold selector with threshold 5%
in order to reduces the number of features. Then we usa a mask (selected_features) in order to keep the selected features

In [174]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(0.001)
reduced_train_data = selector.fit_transform(x_train)
print reduced_train_data.shape

selected_features = selector.get_support()
reduced_test_data = x_test[:,selected_features]

print reduced_test_data.shape

(166, 48)
(42, 48)


## Scaling features
We need to scale our data, so we use a StandardScaler in order to achieve this.

In [175]:
# from sklearn import preprocessing as pr
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_train_data = scaler.fit_transform(reduced_train_data)
scaled_test_data =  scaler.fit_transform(reduced_test_data)

print scaled_train_data.shape


(166, 48)


## Cross Validation

After data preprocessing, it's time to select the suitable MLP Classifier. We will break our train set into 10 sets
and we are going to use 2 metrics: f1_macro and f1_weighted, so we need to make a new scorer which sets the final
score as the sum of the previous metrics' score

In [176]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

def my_scorer(estimator, x, y):
    prediction = estimator.predict(x)
    macro_score = f1_score(y, prediction, average='macro')
    weighted_score = f1_score(y, prediction, average='weighted')
    return macro_score + weighted_score
    
k = 10
scoring_policy = ['f1_macro', 'f1_weighted']




Now it's time to estimate MLP Classifier parameters, using cross validation.
First we have to find the best solve.

In [177]:

solver_opts = ['lbfgs', 'sgd', 'adam']

f_score=[]
leksiko={}
for i in range(0,len(solver_opts)):
    print solver_opts[i]
    clf = MLPClassifier(solver=solver_opts[i])
    clf.fit(scaled_train_data,y_train)              
    val_scores=cross_val_score(clf,x_train,y_train,cv=k,scoring= my_scorer)
    f_score.append( np.mean(val_scores))
    leksiko[solver_opts[i]] = np.mean(val_scores)

f_solver = solver_opts[np.argmax(f_score)]
print f_solver
print f_score
print np.max(f_score)
print leksiko


lbfgs
sgd
adam
lbfgs
[1.6720382451226103, 1.2064612045871299, 1.6586339065131632]
1.67203824512
{'adam': 1.6586339065131632, 'sgd': 1.2064612045871299, 'lbfgs': 1.6720382451226103}


Then we have to find the right alpha value.

In [178]:
alpha_opt =  np.logspace(-5, 3, 5)
print alpha_opt.size
f_score=[]
leksiko={}
for i in range(0,alpha_opt.size):
    print alpha_opt[i]
    clf = MLPClassifier(solver=f_solver,alpha=alpha_opt[i])
    clf.fit(scaled_train_data,y_train)
    val_scores=cross_val_score(clf,scaled_train_data,y_train,cv=k,scoring= my_scorer)
    f_score.append( np.mean(val_scores))
    leksiko[alpha_opt[i]] = np.mean(val_scores)

f_alpha = alpha_opt[np.argmax(f_score)]
print f_alpha
print f_score
print np.max(f_score)

print leksiko

f_clf =  MLPClassifier(solver=f_solver,alpha=f_alpha)
f_clf.fit(scaled_train_data,y_train)

5
1e-05
0.001
0.1
10.0
1000.0
10.0
[1.6838322852757837, 1.7227497665732963, 1.7094273674382034, 1.7331589626403869, 0.69426847662141788]
1.73315896264
{1000.0: 0.69426847662141788, 0.001: 1.7227497665732963, 1.0000000000000001e-05: 1.6838322852757837, 10.0: 1.7331589626403869, 0.10000000000000001: 1.7094273674382034}


MLPClassifier(activation='relu', alpha=10.0, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

Now it's time to find the best combination of hidden layers. The range is (100,0) - (0,100)

## PCA
we will to use PCA method

In [179]:
from sklearn.decomposition import PCA

n = 15
pca = PCA(n_components=n)

trainPCA = pca.fit_transform(scaled_train_data)
testPCA = pca.fit_transform(scaled_test_data)
print scaled_train_data.shape
print trainPCA.shape

print scaled_test_data.shape
print testPCA.shape

results = f_clf.predict(scaled_test_data)

print classification_report(y_test, results)
print accuracy_score(results,y_test)

(166, 48)
(166, 15)
(42, 48)
(42, 15)
             precision    recall  f1-score   support

          0       0.65      0.65      0.65        17
          1       0.76      0.76      0.76        25

avg / total       0.71      0.71      0.71        42

0.714285714286
